# Fire Frequency

This is a draft indicator for use in Southeast Blueprint 2022.




In [1]:
import os
import arcpy

In [1]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
LBAOutputWorkspace = r"E:\projects\BP\Indicators\FireFrequency\working\LBA"
LBAWorkspace = r"E:\generalgis\BurnedArea\LBArasters\tif2process"
MTBSOutputWorkspace = r"E:\projects\BP\Indicators\FireFrequency\working\MTBS"
MTBSWorkspace = r"E:\generalgis\BurnedArea\MTBSrasters"
OutputWorkspace = r"E:\projects\BP\Indicators\FireFrequency\working"

# define final output
SEout = r"e:\projects\BP\Indicators\FireFrequency\working\FireFrequency2.tif"

# define raster used for cell size, extent, and snapping
SERaster= r"E:\projects\BP\1_Extent\BaseBlueprintExtent2022.tif"

# define additional inputs - combination of NLCD and IF max 1=areas that can not burn (water, devevloped, ag, barren, Woody wetlands that are typically wet etc..)
SEfiremask= r"E:\projects\BP\Indicators\FireFrequency\SE_firemask.tif"

## collect source info for burned areas

In [3]:
# create a list of all the rasters in the Landsat Burned Areas (LBA) workspace, including file paths
LBArasters = []
walk = arcpy.da.Walk(LBAWorkspace)
for dirpath, dirnames, filenames in walk:
    for filename in filenames:
        LBArasters.append(os.path.join(dirpath,filename))

In [4]:
# create a list of all the rasters in the MTBS workspace, including file paths
MTBSrasters = []
walk = arcpy.da.Walk(MTBSWorkspace)
for dirpath, dirnames, filenames in walk:
    for filename in filenames:
        MTBSrasters.append(os.path.join(dirpath,filename))

## Process LBA

In [5]:
# Set the source workspace for Landsat Burned Areas (LBA) data
arcpy.env.workspace = LBAWorkspace

In [6]:
print(arcpy.env.workspace)
print(LBArasters)

E:\generalgis\BurnedArea\LBArasters\tif2process
['E:\\generalgis\\BurnedArea\\LBArasters\\tif2process\\LBA_CU_2013_20200415_C01_V01_BF_L8.tif', 'E:\\generalgis\\BurnedArea\\LBArasters\\tif2process\\LBA_CU_2014_20200415_C01_V01_BF_L8.tif', 'E:\\generalgis\\BurnedArea\\LBArasters\\tif2process\\LBA_CU_2015_20200415_C01_V01_BF_L8.tif', 'E:\\generalgis\\BurnedArea\\LBArasters\\tif2process\\LBA_CU_2016_20200415_C01_V01_BF_L8.tif', 'E:\\generalgis\\BurnedArea\\LBArasters\\tif2process\\LBA_CU_2017_20200415_C01_V01_BF_L8.tif', 'E:\\generalgis\\BurnedArea\\LBArasters\\tif2process\\LBA_CU_2018_20200415_C01_V01_BF_L8.tif', 'E:\\generalgis\\BurnedArea\\LBArasters\\tif2process\\LBA_CU_2019_20200415_C01_V01_BF_L8.tif', 'E:\\generalgis\\BurnedArea\\LBArasters\\tif2process\\LBA_CU_2020_20210922_C01_V01_BF_L8.tif', 'E:\\generalgis\\BurnedArea\\LBArasters\\tif2process\\LBA_CU_2021_20220218_C01_V01_BF_L8.tif']


In [10]:
# reclass 2013 to make it consistent with other years - save to LBA working - CONUS
with arcpy.EnvManager(workspace=LBAWorkspace):
    out_raster = arcpy.sa.Con("LBA_CU_2013_20200415_C01_V01_BF_L8.tif", 1, 0, "Value >= 1"); out_raster.save("LBA_CU_2013_20200415_C01_V01_BF_L8.tif")

In [7]:
# use cell statistics to combine data from 2013 - 2021 - SUM ALL CONUS - takes about 20min
out_raster = arcpy.sa.CellStatistics(LBArasters, "SUM", "DATA", "SINGLE_BAND"); out_raster.save("LBABurn2013_2021.tif")

In [8]:
# Clip composite LBA fire rasters to southeast - takes about 20 min
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SERaster, snapRaster=SERaster, cellSize=SERaster):
     LBASE = arcpy.sa.ExtractByMask(out_raster, SERaster); LBASE.save('LBABurn2013_2021_SE.tif');   

In [9]:
# mask with SE firemask
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SERaster, cellSize=SERaster):
    LBASE_masked = arcpy.sa.Con(SEfiremask, 0, LBASE, "Value >= 1"); LBASE_masked.save('LBABurn2013_2021_SE_mask.tif')

## Process MTBS

In [25]:
# Set the source workspace for MTBS data
arcpy.env.workspace = MTBSWorkspace

In [30]:
print(arcpy.env.workspace)


E:\generalgis\BurnedArea\MTBSrasters


In [28]:
# use cell statistics to combine data from 2013 - 2020 - SUM ALL CONUS
MTBS = arcpy.sa.CellStatistics(MTBSrasters, "SUM", "DATA", "SINGLE_BAND"); out_raster.save("MTBSBurn2013_2020.tif")

In [29]:
# clip sum of MTBS to SE
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SERaster, snapRaster=SERaster, cellSize=SERaster):
    MTBS_SE = arcpy.sa.ExtractByMask(MTBS, SERaster); out_raster.save('MTBSBurn2013_2020_SE.tif');

In [11]:
MTBS_LBA = arcpy.sa.CellStatistics(r"E:\generalGIS\BurnedArea\MTBSrasters\MTBSBurn2013_2020_SE.tif;E:\generalGIS\BurnedArea\LBArasters\tif2process\LBABurn2013_2021_SE_mask.tif", "MAXIMUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT"); MTBS_LBA.save(SEout)

In [4]:
# reclassify to create binned version
arcpy.env.workspace = OutputWorkspace
print(arcpy.env.workspace)
out_raster = arcpy.sa.Reclassify(SEout, "Value", "NODATA 0;0 0;1 1;2 2;3 9 3", "DATA"); out_raster.save("FireFreq_indicator.tif")

E:\projects\BP\Indicators\FireFrequency\working


### Finalize indicator

do final steps for all indicators to add description fields, clip and export to SE extent

In [5]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 3:
        return '3 = Burned 3+ times from 2013-2021'
    elif value == 2:
        return '2 = Burned 2 times from 2013-2021'
    elif value == 1:
        return '1 = Burned 1 time from 2013-2021' 
    elif value == 0:
        return '0 = Not burned from 2013-2021 or masked'  
"""

In [7]:
# add and calculate description field to hold indicator values
out_raster = arcpy.management.CalculateField(out_raster, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

In [8]:
# clip to SE 2021 Blueprint extent
#out_raster = arcpy.sa.ExtractByMask("FireFrequency_remask.tif", r"E:\projects\BP\1_Extent\BaseBlueprintExtent2022.tif"); out_raster.save(r"E:\fire v3\Default.gdb\Extract_Fire1")

with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SERaster, snapRaster=SERaster, cellSize=SERaster):
    out_raster4 = arcpy.sa.ExtractByMask(out_raster, SERaster); out_raster4.save(SEout)

In [9]:
# set code block for next step of adding RGB colors
codeblock = """
def Reclass1(Value):
	if Value == 3:
		return 230
	if Value == 2:
		return 253
	if Value == 1:
		return 254
	else:
		return 255
		
def Reclass2(Value):
	if Value == 3:
		return 85
	if Value == 2:
		return 174
	if Value == 1:
		return 230
	else:
		return 255
		
def Reclass3(Value):
	if Value == 3:
		return 13
	if Value == 2:
		return 107
	if Value == 1:
		return 206
	else:
		return 255
		
"""

In [10]:
# calculate Red field
arcpy.management.CalculateField(SEout, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(SEout, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(SEout, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'e:\\projects\\BP\\Indicators\\FireFrequency\\working\\FireFrequency2.tif'>